SMS spqm classification

UPDATED 6 YEARS AGO

PRAVEEN

In [104]:
import pandas as pd

In [105]:
#Кодировка

import chardet

with open('./SMS_train.csv', 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
result

{'encoding': 'Windows-1252', 'confidence': 0.73, 'language': ''}

In [106]:
SMSTrain = pd.read_csv('./SMS_train.csv',encoding='Windows-1252', usecols=['Message_body', 'Label'])

In [107]:
SMSTrain

,Message_body,Label
0,Rofl. Its true to its name,Non-Spam
1,The guy did some bitching but I acted like i'd...,Non-Spam
2,"Pity, * was in mood for that. So...any other s...",Non-Spam
3,Will ü b going to esplanade fr home?,Non-Spam
4,This is the 2nd time we have tried 2 contact u...,Spam
...,...,...
952,hows my favourite person today? r u workin har...,Non-Spam
953,How much you got for cleaning,Non-Spam
954,Sorry da. I gone mad so many pending works wha...,Non-Spam
955,Wat time ü finish?,Non-Spam


Функция нормализации 
использует регулярные выражения и лемматизацию

In [108]:
from nltk.stem import WordNetLemmatizer
stemmer = WordNetLemmatizer()

import re

def normal(str_):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str_)
        
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 

    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)

    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)

    # Converting to Lowercase
    document = document.lower()

    document = stemmer.lemmatize(document)

    return document

In [109]:
buf = {'Spam': 1, 'Non-Spam': 0}

X_Train_buf = list([i[0] for i in SMSTrain.values.tolist()])
Y_Train_buf = list([buf[i[1]] for i in SMSTrain.values.tolist()])


In [110]:
len(X_Train_buf), len(Y_Train_buf)

(957, 957)

Нормализуем все сообщения

In [111]:
def normal_list(l):
    ret = []
    for stroka in l:
        buf = []
        stroka_new = ''
        for word in stroka.split(' '):
            normal_word = normal(word)
            if normal_word not in ['', ' ']:
                buf.append(normal_word)
            stroka_new = re.sub(r'\s+', ' ', ' '.join(buf), flags=re.I)
        ret.append(stroka_new)
    return ret
    

Создание МЕШКА СЛОВ

In [112]:

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=200, min_df=10, max_df=0.6)
X = vectorizer.fit_transform(normal_list(X_Train_buf)).toarray()


Поиск TFIDF

Умножение частоты термина слова на обратную частоту документа

Частота слова:
Term frequency = (Number of Occurrences of a word)/(Total words in the document)

Обратная частота документа:
IDF(word) = Log((Total number of documents)/(Number of documents containing the word))

In [113]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidfconverter = TfidfTransformer()
X = tfidfconverter.fit_transform(X).toarray()

Разбиение на обучающую и тестовые выборки

In [114]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y_Train_buf, test_size=0.2)

Создание модели и ее обучение

In [115]:
from sklearn.linear_model import SGDClassifier

classifier = SGDClassifier()

classifier.fit(X_train, y_train) 

SGDClassifier()

Работа модели

In [116]:
y_pred = classifier.predict(X_test)

Вывод данных о точности

In [117]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[158   7]
 [  4  23]]
              precision    recall  f1-score   support

           0       0.98      0.96      0.97       165
           1       0.77      0.85      0.81        27

    accuracy                           0.94       192
   macro avg       0.87      0.90      0.89       192
weighted avg       0.95      0.94      0.94       192

0.9427083333333334
